![This is an image](Quant-Trading.jpg)

<font size="3">
Please visit our website <a href="https://www.quant-trading.co" target="_blank">quant-trading.co</a> for more tools on quantitative finance and data science.
</font>

In [1]:
# !pip install pandasdmx

## **How to download data from the EUROSTAT database?**


<font size="3"> Eurostat is the statistical office of the European Union. To download data from the EUROSTAT database you can use the python library pandaSDMX. pandaSDMX is an Apache 2.0-licensed Python library that implements SDMX 2.1, a format for exchange of statistical data and metadata used by national statistical agencies, central banks, and international organisations.<br><br>

<font size="3">pandaSDMX can be used to:<br><br>

<font size="3">*Explore the data available from over 20 data providers such as the World Bank, BIS, ILO, ECB, Eurostat, OECD, UNICEF and United Nations. <br><br>

<font size="3">*Parse data and metadata in SDMX-ML (XML) or SDMX-JSON formats. <br><br>

<font size="3">*Convert data and metadata into pandas objects, for use with the analysis, plotting, and other tools in the Python data science ecosystem. <br><br>

<font size="3">…and much more.<br><br>

<font size="3"> You can even use pandaSDMX to download data from the European Central Bank "ECB", The Bank for International Settlement "BIS" and some other organizations. At the end of this notebook we will show an exmaple on how to do that for the BIS data. 


In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandasdmx as pdmx
import datetime

## **Initiate an instance of the specific database**


<font size="3">
In this example we show how to download data for the ESTAT database. Below is an example using the Request method from the pandasDMX library.

In [3]:
#Initialize with ESTAT data
estat = pdmx.Request("ESTAT")
estat

<class 'pandasdmx.api.Request'> instance, source:                ESTAT (Eurostat)

<font size="3">
Then we need to use the method dataflow to get a flow message.

In [4]:
flow_msg = estat.dataflow()
flow_msg

<pandasdmx.StructureMessage>
  <Header>
    id: 'DF1706875049'
    prepared: '2024-02-02T12:57:29.853000+01:00'
    sender: <Agency ESTAT>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (7666): MAR_GO_QM_MT MAR_GO_QM_NL MAR_GO_QM_NO MAR...
  DataStructureDefinition (7216): MAR_GO_QM_MT MAR_GO_QM_NL MAR_GO_QM_N...

<font size="3">
And then, using dataflow again we can check the different indicators contained in the database.

In [5]:
flow_msg.dataflow

{'MAR_GO_QM_MT': <DataflowDefinition ESTAT:MAR_GO_QM_MT(1.0): Gross weight of goods transported to/from main ports - Malta - quarterly data>,
 'MAR_GO_QM_NL': <DataflowDefinition ESTAT:MAR_GO_QM_NL(1.0): Gross weight of goods transported to/from main ports - Netherlands - quarterly data>,
 'MAR_GO_QM_NO': <DataflowDefinition ESTAT:MAR_GO_QM_NO(1.0): Gross weight of goods transported to/from main ports - Norway - quarterly data>,
 'MAR_GO_QM_PL': <DataflowDefinition ESTAT:MAR_GO_QM_PL(1.0): Gross weight of goods transported to/from main ports - Poland - quarterly data>,
 'MAR_GO_QM_PT': <DataflowDefinition ESTAT:MAR_GO_QM_PT(1.0): Gross weight of goods transported to/from main ports - Portugal - quarterly data>,
 'MAR_GO_QM_RO': <DataflowDefinition ESTAT:MAR_GO_QM_RO(1.0): Gross weight of goods transported to/from main ports - Romania - quarterly data>,
 'MAR_GO_QM_SE': <DataflowDefinition ESTAT:MAR_GO_QM_SE(1.0): Gross weight of goods transported to/from main ports - Sweden - quarterly

<font size="3">
It is probably easier to look at them from a pandas DataFrame as shown below:

In [6]:
dataflows = pdmx.to_pandas(flow_msg.dataflow)
dataflows

MAR_GO_QM_MT      Gross weight of goods transported to/from main...
MAR_GO_QM_NL      Gross weight of goods transported to/from main...
MAR_GO_QM_NO      Gross weight of goods transported to/from main...
MAR_GO_QM_PL      Gross weight of goods transported to/from main...
MAR_GO_QM_PT      Gross weight of goods transported to/from main...
                                        ...                        
HLTH_EHIS_PA2U    Self-reported screening of cardiovascular dise...
HLTH_EHIS_PA5E    Self-reported last colorectal cancer screening...
HLTH_EHIS_PA5I    Self-reported last colorectal cancer screening...
HLTH_EHIS_PA5U    Self-reported last colorectal cancer screening...
HLTH_EHIS_PA6E    Self-reported last colonoscopy by sex, age and...
Length: 7666, dtype: object

<font size="3">
As you can see there are a lot of indicators. We can check if there are any related with GDP growth. For that, we can use the function str.contains() 

In [7]:
dataflows[dataflows.str.contains('gdp growth', case=False)].head(30)

TEC00115    Real GDP growth rate - volume
dtype: object

<font size="3">
Fortunately we found one indicator. Its name is "Real GDP growth rate - volume" and it has the identification code "TEC00115". We will use thise code to get the metadata related to it as shown below

In [8]:
metadata = estat.datastructure('TEC00115')
metadata

<pandasdmx.StructureMessage>
  <Header>
    id: 'DSD1706813388'
    prepared: '2024-02-01T18:49:48.756000+00:00'
    sender: <Agency ESTAT>
    source: 
    test: False
  response: <Response [200]>
  Codelist (5): FREQ UNIT NA_ITEM GEO OBS_FLAG
  ConceptScheme (1): TEC00115
  DataStructureDefinition (1): TEC00115

<font size="3">
From the metadata information we can look at the last row, which is called DataStructureDefinition. In this case is the same identification code we found before. We need to use that code to download the data. We can also use the metadata to get the information of the different fields in the database using the method codelist.

In [9]:
metadata.codelist

{'FREQ': <Codelist ESTAT:FREQ(3.2) (11 items): Time frequency>,
 'UNIT': <Codelist ESTAT:UNIT(24.0) (714 items): Unit of measure>,
 'NA_ITEM': <Codelist ESTAT:NA_ITEM(15.0) (665 items): National accounts indicator (ESA 2010)>,
 'GEO': <Codelist ESTAT:GEO(17.0) (4089 items): Geopolitical entity (reporting)>,
 'OBS_FLAG': <Codelist ESTAT:OBS_FLAG(1.33) (46 items): Observation status (Flag)>}

<font size="3">
To download the information we can use the method "data". We can use the identification code we found before, and also a parameter to indicate which is the starting date

In [10]:
resp = estat.data(
    'TEC00115',    
    params={'startPeriod': '2000'},
    )

<font size="3">
We can transform the information into a pandas DataFrame to have a better visualization

In [11]:
data = resp.to_pandas(datetime={'dim': 'TIME_PERIOD', 'freq': 'freq'})
data

geo,AL,AT,BA,BE,BG,CH,CY,CZ,DE,DK,...,NO,PL,PT,RO,RS,SE,SI,SK,TR,UK
na_item,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,...,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ,B1GQ
unit,CLV_PCH_PRE,CLV_PCH_PRE,CLV_PCH_PRE,CLV_PCH_PRE,CLV_PCH_PRE,CLV_PCH_PRE,CLV_PCH_PRE,CLV_PCH_PRE,CLV_PCH_PRE,CLV_PCH_PRE,...,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB,CLV_PCH_PRE_HAB
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
2012,1.4,0.7,-0.8,0.7,0.7,1.2,-3.4,-0.8,0.4,0.2,...,1.4,1.5,-3.7,2.4,-0.2,-1.3,-2.8,1.2,3.5,0.8
2013,1.0,0.0,2.3,0.5,-0.5,1.8,-6.6,0.0,0.4,0.9,...,-0.2,0.9,-0.4,0.6,3.4,0.3,-1.2,0.5,7.1,1.5
2014,1.8,0.7,1.2,1.6,0.9,2.3,-1.8,2.3,2.2,1.6,...,0.9,3.9,1.3,4.5,-1.1,1.6,2.7,2.6,3.5,2.1
2015,2.2,1.0,4.3,2.0,3.4,1.6,3.4,5.4,1.5,2.3,...,0.8,4.5,2.2,3.7,2.3,3.4,2.1,5.1,4.7,1.6
2016,3.3,2.0,3.2,1.3,3.0,2.1,6.6,2.5,2.2,3.2,...,0.3,3.0,2.3,3.5,3.9,0.8,3.1,1.8,1.9,0.9
2017,3.8,2.3,3.2,1.6,2.7,1.4,5.7,5.2,2.7,2.8,...,1.7,5.2,3.8,8.8,2.6,1.2,4.8,2.8,6.1,1.1
2018,4.0,2.4,3.8,1.8,2.7,2.9,5.6,3.2,1.0,2.0,...,0.2,6.0,3.0,6.6,5.1,0.8,4.1,3.9,1.6,0.6


<font size="3">
We can use the pandas xs method to return a specified section of the DataFrame. In this case we will filter by country, selecting only "DE" which stands for Germany

In [12]:
data.xs('DE',  axis=1, drop_level=False)

geo                  DE                
na_item            B1GQ                
unit        CLV_PCH_PRE CLV_PCH_PRE_HAB
TIME_PERIOD                            
2012                0.4             0.2
2013                0.4             0.2
2014                2.2             1.8
2015                1.5             0.6
2016                2.2             1.4
2017                2.7             2.3
2018                1.0             0.7
2019                1.1             0.8
2020               -3.8            -3.9
2021                3.2             3.1
2022                1.8             1.1
2023               -0.3            -1.2

<font size="3">
Here we got the historical dataseries of the GDP growth of Germany. However, we got 2 different dataseries. To explore what they are, we can apply a simple filter using the code below

In [13]:
df  = pdmx.to_pandas(metadata.codelist['UNIT'])
df.reset_index(inplace=True)
df1 = df.loc[(df['UNIT']=='CLV_PCH_PRE_HAB')|(df['UNIT']=='CLV_PCH_PRE')]
df1

,UNIT,name,parent
688,CLV_PCH_PRE,"Chain linked volumes, percentage change on pre...",UNIT
691,CLV_PCH_PRE_HAB,"Chain linked volumes, percentage change on pre...",UNIT


<font size="3">
In the previous code we put the information into a pandas DataFrame first. Then we used the reset_index() method and finally used the "UNIT" column to apply a filter on it. The filter used the or operator | on both names we got. And to identifie the detail of those name we can do the following

In [14]:
print(df1.iloc[0]['name'])
print(df1.iloc[1]['name'])

Chain linked volumes, percentage change on previous period
Chain linked volumes, percentage change on previous period, per capita


## **Download information from the BIS database**


<font size="3">
To download information from the BIS database we can do pretty much the same. Follow the steps we showed before

In [15]:
bis = pdmx.Request("BIS")
flow_msg1 = bis.dataflow()
flow_msg1.dataflow

{'WS_CBPOL_D': <DataflowDefinition BIS:WS_CBPOL_D(1.0): Policy rates daily>,
 'WS_CBPOL_M': <DataflowDefinition BIS:WS_CBPOL_M(1.0): Policy rates monthly>,
 'WS_CBS_PUB': <DataflowDefinition BIS:WS_CBS_PUB(1.0): BIS consolidated banking>,
 'WS_CPMI_CASHLESS': <DataflowDefinition BIS:WS_CPMI_CASHLESS(1.0): CPMI cashless payments (T5-6)>,
 'WS_CPMI_CT1': <DataflowDefinition BIS:WS_CPMI_CT1(1.0): CPMI comparative tables type 1>,
 'WS_CPMI_CT2': <DataflowDefinition BIS:WS_CPMI_CT2(1.0): CPMI comparative tables type 2>,
 'WS_CPMI_DEVICES': <DataflowDefinition BIS:WS_CPMI_DEVICES(1.0): CPMI payment devices>,
 'WS_CPMI_INSTITUTIONS': <DataflowDefinition BIS:WS_CPMI_INSTITUTIONS(1.0): CPMI institutions>,
 'WS_CPMI_MACRO': <DataflowDefinition BIS:WS_CPMI_MACRO(1.0): CPMI Macro>,
 'WS_CPMI_PARTICIPANTS': <DataflowDefinition BIS:WS_CPMI_PARTICIPANTS(1.0): CPMI participants>,
 'WS_CPMI_SYSTEMS': <DataflowDefinition BIS:WS_CPMI_SYSTEMS(1.0): CPMI systems (T8-9-11-13-14-16-17-18-19)>,
 'WS_CPP': <Da

<font size="3">
Here we selected the BIS effective exchange rate daily with identification code "WS_EER_D"

In [16]:
WS_EER_D_msg = bis.dataflow('WS_EER_D')
WS_EER_D_msg

<pandasdmx.StructureMessage>
  <Header>
    id: 'IDREFb132ffdd-9d8b-4aaf-b73b-f1d9457afa40'
    prepared: '2024-02-02T12:02:30+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency UNKNOWN>
    source: 
    test: False
  response: <Response [200]>
  Categorisation (1): 2fa0f830-cd9c-632e-dffd-bceebf593440
  CategoryScheme (1): BISWEB_CATSCHEME
  Codelist (9): CL_AREA CL_BIS_UNIT CL_COLLECTION CL_CONF_STATUS CL_EER...
  ConceptScheme (1): STANDALONE_CONCEPT_SCHEME
  ContentConstraint (2): EER_D_NOM_BRO EER_D_NOM_NAR
  DataflowDefinition (1): WS_EER_D
  DataStructureDefinition (1): BIS_EER
  AgencyScheme (1): AGENCIES

In [17]:
metadata2 = bis.datastructure('BIS_EER')

<font size="3">
From the metadata information we will also get the dimensions. This is a little different than we did before, but we need that information for the BIS database, since the identification code refers to one of the components of the dimensions.

In [18]:
dsd = WS_EER_D_msg.dataflow.WS_EER_D.structure

<font size="3">
Here we can see that one dimension component if REF_AREA. This probably is related to the country. 

In [19]:
dsd.dimensions.components

[<Dimension FREQ>,
 <Dimension EER_TYPE>,
 <Dimension EER_BASKET>,
 <Dimension REF_AREA>,
 <TimeDimension TIME_PERIOD>]

<font size="3">
Let's confirm if that is indeed the case

In [20]:
cl = dsd.dimensions.get('REF_AREA').local_representation.enumerated
pdmx.to_pandas(cl).reset_index().head(30)

,CL_AREA,name,parent
0,1X,ECB,CL_AREA
1,4T,Emerging market economies (aggregate),CL_AREA
2,5A,All reporting economies,CL_AREA
3,5R,Advanced economies,CL_AREA
4,AE,United Arab Emirates,CL_AREA
5,AL,Albania,CL_AREA
6,AR,Argentina,CL_AREA
7,AT,Austria,CL_AREA
8,AU,Australia,CL_AREA
9,BA,Bosnia & Herzegovina,CL_AREA


<font size="3">
Now we are ready to download the information as we did before with the EUROSTAT database. Here we will show that we can download the information for specific values of the indicator. In this case we are only downloading information for 2 countries: Germany and Austria, "DE" and "AT".

In [21]:
key = dict(REF_AREA =['DE', 'AT'])
params = dict(startPeriod='2016')

bis = pdmx.Request('BIS')
data_msg2 = bis.data('WS_EER_D', key=key, params=params)
data = data_msg2.to_pandas(datetime={'dim': 'TIME_PERIOD', 'freq': 'FREQ'})
data

EER_TYPE          N                       
EER_BASKET        N              B        
REF_AREA         DE     AT      DE      AT
TIME_PERIOD                               
2016-01-01    97.71  98.90   94.39   96.72
2016-01-02      NaN    NaN     NaN     NaN
2016-01-03      NaN    NaN     NaN     NaN
2016-01-04    97.82  98.97   94.63   96.87
2016-01-05    97.38  98.73   94.06   96.51
...             ...    ...     ...     ...
2024-01-25   100.02  99.70  102.64  101.28
2024-01-26    99.92  99.64  102.53  101.21
2024-01-27      NaN    NaN     NaN     NaN
2024-01-28      NaN    NaN     NaN     NaN
2024-01-29    99.77  99.54  102.36  101.11

[2951 rows x 4 columns]

If this content is helpful and you want to make a donation please click on the button

[![paypal](https://www.paypalobjects.com/en_US/i/btn/btn_donateCC_LG.gif)](https://www.paypal.com/cgi-bin/webscr?cmd=_s-xclick&hosted_button_id=29CVY97MEQ9BY)